In [80]:
import json
import pandas as pd

In [81]:
import cryptography

In [71]:
from cryptography.hazmat.backends import default_backend
from cryptography.hazmat.primitives.asymmetric import rsa
from cryptography.hazmat.primitives import serialization

In [72]:
from cryptography.hazmat.primitives import hashes
from cryptography.hazmat.primitives.asymmetric import padding

### IMPORT DATA

In [25]:
dataframe = pd.read_csv('data/all-files-ignore-code-kernel-fix.csv', index_col='Unnamed: 0')

In [26]:
data = dataframe.to_dict()

### ENCRYPTION KEY STUFF

In [56]:
# https://nitratine.net/blog/post/asymmetric-encryption-and-decryption-in-python/

In [88]:
def generate_keys():
    
    # PRIVATE KEY
    private_key = rsa.generate_private_key(
        public_exponent=65537,
        key_size=2048,
        backend=default_backend()
    )
    
    # PUBLIC KEY
    public_key = private_key.public_key()
    
    # RETURN
    return public_key, private_key

In [65]:
def save_pubkey(pubkey, filename):
    
    # CREATE PEM
    pem = pubkey.public_bytes(
        encoding=serialization.Encoding.PEM,
        format=serialization.PublicFormat.SubjectPublicKeyInfo
    )

    # CREATE FILE
    with open('data/keys/' + filename, 'wb') as file:
        file.write(pem)

In [66]:
def save_privkey(privkey, filename):
    
    # CREATE PEM
    pem = privkey.private_bytes(
        encoding=serialization.Encoding.PEM,
        format=serialization.PrivateFormat.PKCS8,
        encryption_algorithm=serialization.NoEncryption()
    )

    # CREATE FILE
    with open('data/keys/' + filename, 'wb') as file:
        file.write(pem)

### GENERATE KEYS

In [67]:
global_pubkey, global_privkey = generate_keys()

### CREATE BATCH OF LOG EVENTS

In [43]:
def create_batch(hashes, data):
    
    # DATA CONTAINER
    container = {}
    
    # DATAFRAME COLUMNS
    cols = data.keys()
    
    # LOOP THROUGH
    for target_hash in hashes:
        
        # CREATE ENTRY
        container[target_hash] = {}
        
        # LOOP THROUGH COLUMNS
        for col in cols:
            
            # ADD PROPERTY
            container[target_hash][col] = data[col][target_hash]
    
    # RETURN COLLECTED DATA
    return container

In [44]:
batch = create_batch([
    '53eac99acba6c33cfabdf631e4c7a0196f2e1647a0303190375ea680190510ed',
    '81759ec7de5efa9657b37800e1ec317692fe41c67ca7de759b23fed45015a931',
    '30129d78176e0d7a3e9fe5948b2df6bd3b4afd183f6b47f4c67a585239bf8fd7',
    'c4ec89aae91fea6f4c675192f0d774c5796e101500d50d7db77bcac276ee0a16'
], data)

### ENCRYPT / DECRYPT FILES

In [112]:
def encrypt(data, pubkey):
    
    # CONVERT DICT TO BYTES
    data_bytes = json.dumps(data).encode('utf-8')

    # ENCRYPT WITH PUBKEY
    encrypted = pubkey.encrypt(
        data_bytes,
        padding.OAEP(
            mgf=padding.MGF1(algorithm=hashes.SHA256()),
            algorithm=hashes.SHA256(),
            label=None
        )
    )
    
    return encrypted

In [113]:
def decrypt(data, privkey):

    # DECRYPT WITH PRIVKEY
    decrypted = privkey.decrypt(
        data,
        padding.OAEP(
            mgf=padding.MGF1(algorithm=hashes.SHA256()),
            algorithm=hashes.SHA256(),
            label=None
        )
    )
    
    return decrypted

### ENCRYPT BATCH EVENT

In [97]:
b'encrypt me!'

b'encrypt me!'

In [100]:
json.dumps(batch).encode('utf-8')

b'{"53eac99acba6c33cfabdf631e4c7a0196f2e1647a0303190375ea680190510ed": {"occurrence": 4, "module": "cron", "code": "325", "message": "(CRON) INFO (pidfile fd = 3)", "timestamp": 1613389600}, "81759ec7de5efa9657b37800e1ec317692fe41c67ca7de759b23fed45015a931": {"occurrence": 6, "module": "CRON", "code": "15203", "message": "(CRON) info (No MTA installed, discarding output)", "timestamp": 1613646001}, "30129d78176e0d7a3e9fe5948b2df6bd3b4afd183f6b47f4c67a585239bf8fd7": {"occurrence": 144, "module": "CRON", "code": "6126", "message": "(root) CMD ( cd / && run-parts --report /etc/cron.hourly)", "timestamp": 1613625421}, "c4ec89aae91fea6f4c675192f0d774c5796e101500d50d7db77bcac276ee0a16": {"occurrence": 6, "module": "CRON", "code": "15207", "message": "(root) CMD (./home/wickstjo/scripts/ropsten.sh)", "timestamp": 1613646001}}'

In [ ]:
encrypt(batch, global_pubkey)